In [ ]:
import pandas as pd
import numpy as np
import re

# Funções utilizadas

In [ ]:
# @title Criando o dataframe
# Inicializando os vetores que serão utilizados para receber os times nas respectivas fases
top_f3 = []
top_f4 = []
campeao = []
vice = []
finalistas = []
participantes = []
reb = []
n_reb = []
ultimos = []
# Colunas do Dataframe
colunas_df = ['Time', 'COP_B_1', 'COP_B_2', 'COP_B_4', 'COP_B_8', 'COP_B_16', 'COP_B_P']
df_copa = pd.DataFrame(columns=colunas_df)
df_copa

,Time,COP_B_1,COP_B_2,COP_B_4,COP_B_8,COP_B_16,COP_B_P


Dicionário de colunas

| Nome | Descrição |
|------|-----------|
| COP_B_1 | Vencedor da edição |
| COP_B_2 | Vice da Edição |
| COP_B_4 | 3º e 4º lugar |
| COP_B_8 | 5º ao 8º |
| COP_B_16 | 9º ao 16º |
| COP_B_P | Número de participações |

In [ ]:
# @title Função limpar nomes
# Usando regex para limpar o '(x)' no final do nome do time
def clean_text(text):
    text = pd.Series(text).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
    # Remove espaços em branco no início e no final
    return text.str.strip()

In [ ]:
# @title Função get Bracket
# Esta função pega o chaveamento da copa na Wikipedia
def get_bracket(final, bracket):
  top_4 = []
  top_8 = []
  bracket = bracket.loc[2:, :]
  participantes = final
  print(f'Finalistas: {final}')

  # A partir do final do Chaveamento anda as colunas até a fase inicial para recuperar os times
  for i in range(len(bracket.columns)-1, 0, -1):

      # Pegar os times sem repetição na coluna desejada
      times = bracket.loc[:, i].dropna().unique()
      # Retira os possíveis () e []
      times = pd.Series(times).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
      # Define o tipo como String
      times = times.astype(str)
      # Garante que o conteúdo não contém apenas números
      times = times[~times.str.match(r'^[0-9\s]+$')]
      times = times.to_list()
      # Limpa o conteúdo
      times = [team.strip() for team in times if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)]

      # Se a quantidade de times retornada for igual a 4, quer dizer que a coluna é a semifinal
      if (len(times) == 4) & (len(top_4) == 0):
          # Recupera os times que ainda não estão na lista 'participantes'
          top_4 = [team for team in times if team not in participantes]
          print(f'Semis:  {top_4}')
          # Adiciona os times da semifinal à lista 'participantes'
          participantes = set(participantes).union(top_4)

      elif (len(times) == 8) & (len(top_8) == 0): # 8 times = coluna Quartas de final
          top_8 = [team for team in times if team not in participantes]
          print(f'Quartas: {top_8}')
          participantes = set(participantes).union(top_8)

  return participantes, top_4, top_8

In [ ]:
# @title Função get Table
def get_table(table, ano):
  #dftemp = dftemp.rename(columns={'Equipevde': 'Time'})
  tabela = table.rename(columns={table.columns[0]: 'Pos', table.columns[1]: 'Time'})
  #dftemp['Pos'] = dftemp['Pos'].astype(int)
  campeao = []
  vice = []
  top_4 = []
  top_8 = []
  top_16 = []
  for i, row in tabela.iterrows():
    pos = float(row["Pos"])
    time = clean_text(row["Time"]).to_list()
    #last_column_value = list(getattr(row, '_asdict')().values())[-1]
    last_column_value = row.iloc[table.shape[1]-1]
    if pos == 1:
      campeao = time
    elif pos == 2:
      vice = time
    elif (pos <= 4):
      top_4 = top_4 + time
    elif (pos <= 8):
      top_8 = top_8 + time
    elif (pos <= 16):
      top_16 = top_16 + time

  print(f'Participantes: {len(tabela)}')
  print(f'Campeão: {campeao}')
  print(f'Vice: {vice}')
  print(f'3º e 4º: {top_4}')
  print(f'5º ao 8º: {top_8}')
  print(f'9º ao 16º: {top_16}')

  participantes = campeao + vice + top_4 + top_8 + top_16

  return participantes

In [ ]:
# @title Função get Table com Beautiful Soup
def get_table_bs(url, t):
  import requests
  from bs4 import BeautifulSoup

  wikiurl = url
  table_class = "wikitable sortable jquery-tablesorter"
  response = requests.get(wikiurl)
  # 200
  soup = BeautifulSoup(response.text, 'html.parser')
  tables_all = soup.find_all('table', {"class": "wikitable"})

  # <table class="wikitable sortable">
  # <tbody><tr>
  # <th>Name of Town
  # </th>
  # <th>State
  # ....

  df = pd.read_html(str(tables_all[t]))
  return df

In [ ]:
# @title Função get times das fases preliminares
# Esta função recupera os times que foram eliminados nas fases preliminares ou
# em uma fase cuja tabela tem as colunas: Equipe 1 | Resultado | Equipe 2
def get_times_preliminares(df, list_todos):
  df_pre = pd.concat([df['Equipe 1'], df['Equipe 2']], ignore_index=True)
  top = df_pre.astype(str)
  top = top.replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
  top = top[~top.str.match(r'^[0-9\s]+$')]
  top = top.to_list()
  top = [team.strip() for team in top
         if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)
         and team not in list_todos]
  return top

In [ ]:
# @title Função preencher tabela
def preencher_tabela(times_ed, df):
  times_tabela = df['Time'].to_list()
  # Itera sobre os participantes da edição
  for pos, team in enumerate(times_ed):
      pos += 1
      # Insere o time caso seja a primeira participação
      if team not in times_tabela:
          print(f'Novo participante: {team}')
          df.loc[len(df)] = [team] + [0] * (len(df.columns) - 1)
          # df.loc[len(df)]

      if pos == 1:
          df.loc[df['Time'] == team, 'COP_B_1'] += 1
      elif pos == 2:
          df.loc[df['Time'] == team, 'COP_B_2'] += 1
      elif (pos <= 4):
          df.loc[df['Time'] == team, 'COP_B_4'] += 1
      elif (pos <= 8):
          df.loc[df['Time'] == team, 'COP_B_8'] += 1
      elif (pos <= 16):
          df.loc[df['Time'] == team, 'COP_B_16'] += 1

      if team in ultimos:
          df.loc[df['Time'] == team, 'COP_B_ULT'] += 1

      # Adiciona a participação do time na edição
      df.loc[df['Time'] == team, 'COP_B_P'] += 1

  return df

# 2000-2002: 28 e 26 Times - Eliminatório


In [ ]:
# 2000: t=27, drop(-1)
# 2001: t=35, drop(-1)
# 2002: t=19, drop(-1)
for i in range(2000, 2002+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_dos_Campe%C3%B5es_de_' + ano)
  print(f'\nCopa de {i}:')
  if (i == 2000):
    t = 27
  elif (i == 2001):
    t = 35
  else:
    t = 19
  df = df[t]
  df.columns = df.columns.droplevel(0)
  df.drop(df.index[-1], inplace=True)
  participantes = get_table(df, int(ano))
  df_copa = preencher_tabela(participantes, df_copa)


Copa de 2000:
Participantes: 9
Campeão: ['Palmeiras']
Vice: ['Sport']
3º e 4º: ['Flamengo', 'São Paulo']
5º ao 8º: ['Goiás', 'América Mineiro', 'Vitória', 'Cruzeiro']
9º ao 16º: ['São Raimundo-AM']
Novo participante: Palmeiras
Novo participante: Sport
Novo participante: Flamengo
Novo participante: São Paulo
Novo participante: Goiás
Novo participante: América Mineiro
Novo participante: Vitória
Novo participante: Cruzeiro
Novo participante: São Raimundo-AM

Copa de 2001:
Participantes: 9
Campeão: ['Flamengo']
Vice: ['São Paulo']
3º e 4º: ['Coritiba', 'Cruzeiro']
5º ao 8º: ['São Raimundo-AM', 'Sport', 'Corinthians', 'Bahia']
9º ao 16º: ['Goiás']
Novo participante: Coritiba
Novo participante: Corinthians
Novo participante: Bahia

Copa de 2002:
Participantes: 16
Campeão: ['Paysandu']
Vice: ['Cruzeiro']
3º e 4º: ['Flamengo', 'Palmeiras']
5º ao 8º: ['Goiás', 'Bahia', 'Fluminense', 'Vitória']
9º ao 16º: ['São Paulo', 'São Caetano', 'Vasco da Gama', 'Náutico', 'Corinthians', 'Grêmio', 'Atlétic

In [ ]:
print(f'Participantes únicos: {len(df_copa)}')

Participantes únicos: 20


# Corrigindo nomes dos times

1. Atlético Paranaense -> Athletico Paranaense

In [ ]:
print('Times diferentes: ', len(df_copa['Time'].sort_values().to_list()))
df_copa['Time'].sort_values().to_list()

Times diferentes:  20


['América Mineiro',
 'Atlético Mineiro',
 'Atlético Paranaense',
 'Bahia',
 'Corinthians',
 'Coritiba',
 'Cruzeiro',
 'Flamengo',
 'Fluminense',
 'Goiás',
 'Grêmio',
 'Náutico',
 'Palmeiras',
 'Paysandu',
 'Sport',
 'São Caetano',
 'São Paulo',
 'São Raimundo-AM',
 'Vasco da Gama',
 'Vitória']

In [ ]:
substituicoes = {
    "Atlético Paranaense": "Athletico Paranaense"
}
df_copa['Time'] = df_copa['Time'].replace(substituicoes)
df_grouped = df_copa.groupby('Time', as_index=False)[colunas_df[1:]].sum()
len(df_grouped['Time'].sort_values().to_list())

20

In [ ]:
print('Total de times diferentes: ', len(df_grouped['Time'].sort_values().to_list()))
df_grouped['Time'].sort_values().to_list()

Total de times diferentes:  20


['América Mineiro',
 'Athletico Paranaense',
 'Atlético Mineiro',
 'Bahia',
 'Corinthians',
 'Coritiba',
 'Cruzeiro',
 'Flamengo',
 'Fluminense',
 'Goiás',
 'Grêmio',
 'Náutico',
 'Palmeiras',
 'Paysandu',
 'Sport',
 'São Caetano',
 'São Paulo',
 'São Raimundo-AM',
 'Vasco da Gama',
 'Vitória']

In [ ]:
df_grouped.head()

,Time,COP_B_1,COP_B_2,COP_B_4,COP_B_8,COP_B_16,COP_B_P
0,América Mineiro,0,0,0,1,0,1
1,Athletico Paranaense,0,0,0,0,1,1
2,Atlético Mineiro,0,0,0,0,1,1
3,Bahia,0,0,0,2,0,2
4,Corinthians,0,0,0,1,1,2


# Pontuação

Como a Copa dos Campeões garantia uma vaga para a Libertadores, considera-se como uma Copa do Brasil Secundária, valendo metade dos pontos.

| Resultado | Pontos  | Pct %   |
|-----------|---------|---------|
| 1º Lugar  | 1150.00 | 100.00% |
| 2º Lugar  |  383.33 |  33.33% |
| 3º e 4º   |  127.78 |  11.11% |
| 5º ao 8º  |   63.89 |   5.56% |
| 9º ao 16º |   38.33 |   3.33% |







In [ ]:
df_rank = df_grouped.sort_values(by=colunas_df[1:], ascending=False).reset_index(drop=True)
df_rank.index += 1

In [ ]:
# Pontuação em relação apenas ao Campeonato Brasileiro
titulo = 100
df_rank['COP_B_PTS'] = round(df_rank['COP_B_1']*titulo +
                  df_rank['COP_B_2']*titulo/3 +
                  df_rank['COP_B_4']*titulo/9 +
                  df_rank['COP_B_8']*titulo/18 +
                  df_rank['COP_B_16']*titulo/30
, 2)

In [ ]:
df_rank = df_rank.sort_values(by=['COP_B_PTS', 'Time'], ascending=[False, True]).reset_index(drop=True)
df_rank.index += 1

# Ranking Geral

In [ ]:
df_rank

,Time,COP_B_1,COP_B_2,COP_B_4,COP_B_8,COP_B_16,COP_B_P,COP_B_PTS
1,Flamengo,1,0,2,0,0,3,133.33
2,Palmeiras,1,0,1,0,0,2,116.67
3,Paysandu,1,0,0,0,0,1,100.00
4,Cruzeiro,0,1,1,1,0,3,55.56
5,São Paulo,0,1,1,0,1,3,53.33
6,Sport,0,1,0,1,0,2,38.89
7,Coritiba,0,0,1,0,0,1,16.67
8,Goiás,0,0,0,2,1,3,14.44
9,Bahia,0,0,0,2,0,2,11.11
10,Vitória,0,0,0,2,0,2,11.11


In [ ]:
# Exportar para Excel
df_rank.to_excel('Copa dos Campeões Rank.xlsx')